In [110]:
"""
Created on Wed Oct  3 14:47:19 2018

@author: Jessie
"""
%matplotlib inline
from __future__ import print_function
import pandas as pd
import numpy as np
import scipy.stats as stats
import sklearn
import random
import os
from sklearn.linear_model import *
import matplotlib.pyplot as plt
from IPython.display import Image
from sklearn.model_selection import train_test_split
from sklearn.metrics import average_precision_score, accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report 
#import statsmodels.api as sm
#import statsmodels.formula.api as smf
#import statsmodels.stats.api as sms

In [46]:
#import numpy array
def LoadSplitData(split):
    DATA_DIR = "Data"
    ENCODING_DIR = os.path.join(DATA_DIR, "binEncoding.csv")
   
    data_xy = pd.read_csv(ENCODING_DIR)
    data_xy = data_xy.drop(columns= ['Unnamed: 0'])

    #drop the target variables you do not need (1-7), market moving or not ratings
    colsxy= [0,1,2,4,5,6] #needs change
    data_xy = data_xy.drop(data_xy.columns[colsxy],axis=1)
    
    cols =[0,1,2,3,4,5,6]
    X = data_xy.drop(data_xy.columns[cols],axis=1)
    y = data_xy.iloc[:,3] #needs change
    
    
    #Get dummy test/train set 
    DummyX_train, DummyX_test, Dummyy_train, Dummyy_test = train_test_split(data_xy, y, test_size=split, random_state=42)
    
    return X, y, DummyX_train, DummyX_test, Dummyy_train, Dummyy_test, data_xy
    


In [47]:
X, y, DummyX_train, DummyX_test, Dummyy_train, Dummyy_test, data_xy = LoadSplitData(0.30) #Decide the test size (% of total data)

In [113]:
#Use Logistic Regression - Testing with dummy-y-variable
def SingleTest(X_train, y_train, X_test, y_test):
    #define classifier
    logReg = LogisticRegression(penalty='l2', dual=False, tol=0.0001, C=1.0, fit_intercept=True, random_state=0, solver='liblinear')

    #Test 
    clfSingleTest = logReg.fit(X_train, y_train)

    # predict on train and test set
    y_train_predict = clfSingleTest.predict(X_train)
    y_test_predict = clfSingleTest.predict(X_test)

    #Binary metrics
    columns = ['Precision','Recall', 'F1', 'Avg Precision', 'Accuracy']
    df = pd.DataFrame(index=['Train','Test'], columns=columns)
    
    
    TrainPrecision = precision_score(y_train, y_train_predict)
    TestPrecision = precision_score(y_test, y_test_predict)
    
    TrainRecall = accuracy_score(y_train, y_train_predict)
    TestRecall = accuracy_score(y_test, y_test_predict)
    
    Trainf1 = f1_score(y_train, y_train_predict, average='binary')
    Testf1 = f1_score(y_test, y_test_predict, average='binary')
    
    #Not to be confused with the ranking metric, mAP (mean average precision), this is simply the average of the P and R curve
    TrainAvgP = average_precision_score(y_train, y_train_predict)
    TestAvgP = average_precision_score(y_test, y_test_predict)
    
    TrainAccuracy = accuracy_score(y_train, y_train_predict)
    TestAccuracy = accuracy_score(y_test, y_test_predict)
    
    df.loc['Train'] = pd.Series({'Precision': TrainPrecision, 'Recall': TrainRecall, 'F1': Trainf1, 'Avg Precision': TrainAvgP, 'Accuracy': TrainAccuracy})
    df.loc['Test'] = pd.Series({'Precision': TestPrecision, 'Recall': TestRecall, 'F1': Testf1, 'Avg Precision': TestAvgP, 'Accuracy': TestAccuracy})
    return df

In [114]:
#Sanity check with dummy variable:  output should be 1.0)
df = SingleTest(DummyX_train, Dummyy_train, DummyX_test, Dummyy_test)
df.head()

,Precision,Recall,F1,Avg Precision,Accuracy
Train,1,1,1,1,1
Test,1,1,1,1,1


In [116]:
def SequentialSetRun(X, y, testsize):
   
    #Predicting on Real DataSet - Only 1 run
    num_articles = len(X) -1 #Subtract header row
    #testsize = 0.30
    trainsize = 1-testsize


    #Select first 70% as train
    X_train = X.iloc[:round(num_articles*trainsize)]
    y_train = y.iloc[:round(num_articles*trainsize)]

    #Followiing 30% is test
    X_test = X.iloc[(round(num_articles*trainsize)):]
    y_test = y.iloc[(round(num_articles*trainsize)):]
   
    #Run SingleTest
    TestResults = SingleTest(X_train, y_train, X_test, y_test)


    return TestResults
    

In [117]:
dfSeqTest = SequentialSetRun(X, y, 0.30) #final number is % size of testing set
dfSeqTest.head()

,Precision,Recall,F1,Avg Precision,Accuracy
Train,0.840249,0.848382,0.819838,0.758562,0.848382
Test,0.59375,0.672619,0.535211,0.487676,0.672619


## The code below is a WIP for potential kfold CV with sequential splits
We can circle back to it if time allows, as we do not anticipate a different result from a single split in data

In [ ]:
#Predicting on Real DataSet - Returns mean accuracy and CI @95%

#We want to split the data into sequential folds

#i=0 #k=2
def kSequential_Split(k, itr, X, data_xy):
    newdata = data_xy
    for row in newdata.index: #where X is our dataset
        #if it is, drop the row X
        if (row % k) == itr:
            #make sure there is a row to drop
            if row <= len(newdata.index):
                #print('Dropping article:', row)
                newdata = newdtata.drop(newdata.index[row])
    XData = newdata.drop(columns=['whatever_youd_like'])  
    yData = newdata['whatever_youd_like']
    return XData, yData

In [ ]:
#We now have our subset of data, newX
#split into 70/30 (first 70 to next 30 chunk of data)
newX.head()
len(newX.index)
newX.shape


In [ ]:
logReg = LogisticRegression(penalty='l2', dual=False, tol=0.0001, C=1.0, fit_intercept=True, random_state=0, solver='liblinear')
clf = logReg.fit(X_train, y_train)

#predict on all rows=articles from row 2 onwards
pred_train = clf.predict(X_train.values[:2,:]) 
pred_test = clf.predict(X_test.values[:2,:])

In [ ]:
#Without Dummy Variable
TrainScores = clf.score(X_train,y_train)
TestScores = clf.score(X_test,y_test)

#Return the mean accuracy on the given test data and labels (should be 1.0)
print(TrainScores)
print(TestScores)

In [ ]:
#Code for k-fold cross validation with random splits. We want sequential splits
def random_mean_ci(X, y, data_xy, num_tests):
    # train_results is a list of train accuracy results for the differrent random splits of the dataset
    train_results = []
    
    # test_results is a list of test accuracy results for the differrent random splits of the dataset
    test_results = []
    
    # Write your code here
    for i in range(num_tests):
        
        #METHOD 1: Random Datasplit (old version - not sequential date sampling)
        #X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=random.randint(1,1000))

        #METHOD 2: Sequential Datasplit Variation of k-fold Cross Valid. (new version - sequential date sampling)
        Xdata, ydata = kSequential_Split(num_tests, i, X, data_xy)
        
        
        # train a logistic regression classifier
        clf = LogisticRegression(C=1.0, penalty='l2', dual=False, tol=0.0001, fit_intercept=True, solver='liblinear').fit(X_train, y_train)

        # predict on train and test set
        y_train_predict = clf.predict(X_train)
        y_test_predict = clf.predict(X_test)

        # calculate train and test accuracy
        train_accuracy = accuracy_score(y_train, y_train_predict)
        test_accuracy = accuracy_score(y_test, y_test_predict)

        # report results, update the results array
        train_results.append(train_accuracy)
        test_results.append(test_accuracy)

    # calculate the train mean and the 95% confidence interval for the list of results
    train_mean = np.mean(train_results)
    train_ci_low, train_ci_high = stats.t.interval(0.95, len(train_results)-1, loc=train_mean, scale=stats.sem(train_results))

    # calculate the test mean and the 95% confidence interval for the list of results
    test_mean = np.mean(test_results)
    test_ci_low, test_ci_high = stats.t.interval(0.95, len(test_results)-1, loc=test_mean, scale=stats.sem(test_results))
    
    # validate return types
    assert isinstance(train_mean, float) and isinstance(train_ci_low, float) and isinstance(train_ci_high, float), "return types"
    assert isinstance(test_mean, float) and isinstance(test_ci_low, float) and isinstance(test_ci_high, float), "return types"
    
    return train_mean, train_ci_low, train_ci_high, test_mean, test_ci_low, test_ci_high

In [ ]:
#Accuracy and CI over 10 runs
train_mean, train_low, train_high, test_mean, test_low, test_high = random_mean_ci(X, y, data_xy, num_tests = 10)
print("Train mean accuracy over 10 random splits: {}".format(train_mean))
print("Train confidence interval over 10 random splits: [{}, {}]".format(train_low, train_high))
print("Test mean accuracy over 10 random splits: {}".format(test_mean))
print("Test confidence interval over 10 random splits: [{}, {}]".format(test_low, test_high))

An initial run with Retail only Articles, to determine market moving or not (Randomly split):

Train mean accuracy over 10 random splits: 0.7402896081771722
Train confidence interval over 10 random splits: [0.7288090750212393, 0.751770141333105]
Test mean accuracy over 10 random splits: 0.5650793650793651
Test confidence interval over 10 random splits: [0.547570102462968, 0.5825886276957621]
